In [1]:
data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [3]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [4]:
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=0.5,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/ablation/delay/0_15",
    clear_cache=False,
)

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


In [5]:
try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=multiprocessing.cpu_count() - 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

2024-12-20 20:02:39,957	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [05:10<00:00,  2.30s/it]


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0850, Top 1: 0.1380, Top 5: 0.3555, Top 10: 0.4875, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0760, Top 1: 0.1123, Top 5: 0.3554, Top 10: 0.5241, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0663, Top 1: 0.1213, Top 5: 0.3387, Top 10: 0.4888, Perplexity: 0.0000
Epoch 1 completed in 5.56m. 0.04m per recording.


Training Epoch 2: 100%|██████████| 135/135 [04:19<00:00,  1.92s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.1150, Top 1: 0.1835, Top 5: 0.4300, Top 10: 0.5530, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1001, Top 1: 0.1472, Top 5: 0.4131, Top 10: 0.5891, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0900, Top 1: 0.1450, Top 5: 0.3862, Top 10: 0.5388, Perplexity: 0.0000
Epoch 2 completed in 4.67m. 0.03m per recording.


Training Epoch 3: 100%|██████████| 135/135 [04:17<00:00,  1.91s/it]


...

Testing at epoch 24
Test unseen_subject completed. Accuracy: 0.1125, Top 1: 0.1725, Top 5: 0.3920, Top 10: 0.5155, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1136, Top 1: 0.1626, Top 5: 0.4182, Top 10: 0.5698, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1225, Top 1: 0.1787, Top 5: 0.3625, Top 10: 0.4825, Perplexity: 0.0000
Epoch 24 completed in 4.66m. 0.03m per recording.


Training Epoch 25: 100%|██████████| 135/135 [04:18<00:00,  1.91s/it]


Testing at epoch 25
Test unseen_subject completed. Accuracy: 0.1025, Top 1: 0.1780, Top 5: 0.4190, Top 10: 0.5430, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1257, Top 1: 0.1790, Top 5: 0.4492, Top 10: 0.6127, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1025, Top 1: 0.1562, Top 5: 0.3425, Top 10: 0.4775, Perplexity: 0.0000
Epoch 25 completed in 4.66m. 0.03m per recording.


Training Epoch 26: 100%|██████████| 135/135 [04:18<00:00,  1.92s/it]


Testing at epoch 26
Test unseen_subject completed. Accuracy: 0.1090, Top 1: 0.1860, Top 5: 0.4015, Top 10: 0.5255, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1331, Top 1: 0.1853, Top 5: 0.4535, Top 10: 0.6015, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1275, Top 1: 0.1850, Top 5: 0.3862, Top 10: 0.4938, Perplexity: 0.0000
Epoch 26 completed in 4.66m. 0.03m per recording.
Early stopping at epoch 26. Highest top 10 accuracy at epoch 15.
Training completed.
unseen_subject: Acc: 0.1685, Top 1: 0.2565, Top 5: 0.4860, Top 10: 0.6115
unseen_task: Acc: 0.2421, Top 1: 0.3220, Top 5: 0.6258, Top 10: 0.7513
unseen_both: Acc: 0.1963, Top 1: 0.2675, Top 5: 0.5200, Top 10: 0.6350


In [6]:
session.highest_epoch

15